#  数据获取

## 数据爬取

### 遗漏景点url

In [ ]:
#出错的链接去重并排序
#listerror为进一步需要爬取的景点
import re
fwa=open('D:\\sharing bikes\\TA Crawl\\listerror.txt','a')
list1=[]
with open('D:\\sharing bikes\\TA Crawl\\list-error.txt','r')as f:
    for line in f:
        list1.append(line)
xa=re.compile('-d(.+?)-')
set1=list(set(list1))
set1.sort(key=lambda x:int(float(re.search(xa,x).group(1))),reverse=False)
for line in set1:
    fwa.write(line)
fwa.close()

In [ ]:
#将有遗漏的文件移出去
#reviewerror为有遗漏的景点，也就是爬取出错的景点
import shutil
import os
originfilelist=os.listdir('D:\\sharing bikes\\TA Crawl\\review')
listerror=[]
xa=re.compile('-d(.+?)-')
with open('D:\\sharing bikes\\TA Crawl\\listerror.txt','r')as f:
    for line in f:
       listerror.append(re.search(xa,line).group(1)) 
for i in listerror:
    if i in originfilelist:
        shutil.move("D:\\sharing bikes\\TA Crawl\\review\\"+i,"D:\\sharing bikes\\TA Crawl\\reviewerror")  
set(listerror)-set(os.listdir('D:\\sharing bikes\\TA Crawl\\reviewerror'))#{'534489'}
#{'534489'}为出错了但没有爬取到任何文件的景点

In [ ]:
#得出遗漏的链接供下一步爬取
datalisthomepage=[]
with open('D:\\sharing bikes\\TA Crawl\\list.txt','r')as fa:
    for line in fa:
        datalisthomepage.append(line)
datalistscrawler=[]
with open('D:\\sharing bikes\\TA Crawl\\list2all.txt','r')as fb:
    for line in fb:
        datalistscrawler.append(line)
datanotsc=set(datalisthomepage)-set(datalistscrawler)
datanotsc
with open('D:\\sharing bikes\\TA Crawl\\notscrawl.txt','a') as da:
    for line in datanotsc:
        da.write(line)

### 遗漏景点url读取

In [ ]:
#因为Java思路有遗漏，所以python重写程序 
#读取并放到生成器中
def ignore_attr():
    with open('D:\\sharing bikes\\TA Crawl\\listerror.txt','r') as f:
        for i in f:
            yield i    

### 数据爬取

# 数据解析

## 爬取结果读取

In [15]:
#读取结果并将结果用生成器方式输出
import os
from bs4 import BeautifulSoup
base='D:\\sharing bikes\\TA Crawl\\review'
listdir=os.listdir(base)
def getpagedata():
    for dir in listdir:
        skey=dir
        listfile=os.listdir(base+'\\'+dir)
        if listfile:
            for file in listfile:
                with open(base+'\\'+dir+'\\'+file,'rb') as f:              
                    yield {skey:f.read()}

## 数据库字段及存储函数

In [19]:
#连接数据库并创建数据表
#创建数据表
import pymysql
db=pymysql.connect(host='localhost',user='root',password='267708',port=3306,db='spider')
cursor=db.cursor()
sql='''
create table if not exists ta_commentfinal(
attraction_id varchar(30) not null,#景点id
comment_id varchar(30) not null,#评论id
user_name varchar(100),#评论用户的名字
user_location varchar(100),#用户定位
user_level varchar(20),#用户在tripadvisor的等级
user_review_counts varchar(20),#用户总评论数
user_attrction_review_counts varchar(20),#用户总的景点评论数
user_votes varchar(20),#用户被点赞总数
review_time varchar(20),#改评论时间
score varchar(20),#评论分数（满意度打分）
comment_topic varchar(200),#评论的标题
comment_content varchar(10000),#评论内容
comment_thanks varchar(10),#此评论被点赞的数目
visited_time varchar(30),#
picture_count varchar(10),
primary key(attraction_id,comment_id))
'''
cursor.execute(sql)
db.close()

In [21]:
import pymysql
db=pymysql.connect(host='localhost',user='root',password='267708',port=3306,db='spider')
cursor=db.cursor()
def save_to_database(data):
    table='ta_commentfinal'
    keys=', '.join(data.keys())
    values=', '.join(['%s']*len(data))
    sql='insert into {table}({keys}) values ({values})on duplicate key update'.format(table=table,keys=keys,values=values)
    update=','.join([" {key}=%s".format(key=key) for key in data])
    sql+=update
    try:
        if cursor.execute(sql,tuple(data.values())*2):
            db.commit()
    except:
        print('Failed')
        db.rollback()

## 文件解析并保存

In [22]:
import re
from bs4 import BeautifulSoup
for data in getpagedata():
    for key,value in data.items():
        soup=BeautifulSoup(value,'lxml')
        attraction=key
        #只能嵌套查询，因为有些字段缺失的原因
        rec=re.compile('(\d+)')
        datapart=soup.find_all(id=re.compile('expanded_review_'))
        for data in datapart:
            attraction_id=attraction
            commentid=re.search('UR(\d+)',data.a['name']).group(1)
            username=data.find(class_=re.compile('expand_inline scrname mbrName'))
            if username:
                username=username.string.strip()
            location=data.find(class_=re.compile('location'))
            if location:
                userlocation=location.string
            userlevel=data.find(class_=re.compile('bold badge lvl_'))
            if userlevel:
                userlevel=userlevel['class'][2][-2:]
            user_review_counts=data.find(class_="reviewerBadge badge")
            if user_review_counts:
                user_review_counts=user_review_counts.span.string
                user_review_counts=re.search(rec,user_review_counts).group(1)
            user_attrction_review_counts=data.find(class_="contributionReviewBadge badge")
            if user_attrction_review_counts:
                user_attrction_review_counts=user_attrction_review_counts.span.string
                user_attrction_review_counts=re.search(rec,user_attrction_review_counts).group(1)
            user_votes=data.find(class_="helpfulVotesBadge badge no_cpu")
            if user_votes:
                user_votes=user_votes.span.string
                user_votes=re.search(rec,user_votes).group(1)
            review_time=data.find(class_="ratingDate relativeDate")
            if review_time:
                review_time=review_time['title']
            else:
                review_time=data.find(class_="ratingDate").string
                review_time=re.search('Reviewed (.*?)\n$',review_time).group(1)
            score=data.find(class_=re.compile("ui_bubble_rating bubble_"))['class']
            comment_topic=data.find(class_="quote")
            if comment_topic:
                comment_topic=comment_topic.a.span.string
            comment_content=data.find(name='p').get_text('','<br>')
            if comment_content:
                comment_content=comment_content.strip()
            comment_thanks=data.find(class_="numHlp")
            if comment_thanks:
                comment_thanks=comment_thanks.span.string.strip()
                comment_thanks=re.search(rec,comment_thanks).group(1)   
            visited_time=data.find(class_="recommend-titleInline")
            if visited_time:
                visited_time=visited_time.string
            picture_count=data.find(class_="photosInline")
            if picture_count:
                picture_count=len(picture_count.ul.find_all(name='li'))
            dataitem={
            'attraction_id':attraction_id,
            'comment_id':commentid,
            'user_name':username,
            'user_location':userlocation,
            'user_level':userlevel,
            'user_review_counts':user_review_counts,
            'user_attrction_review_counts':user_attrction_review_counts,
            'user_votes':user_votes,
            'review_time':review_time,
            'score':int(float(re.search(rec,score[1]).group(1)))/10,
            'comment_topic':comment_topic,
            'comment_content':comment_content,
            'comment_thanks':comment_thanks,
            'visited_time':visited_time,    
            'picture_count':picture_count
            }
            save_to_database(dataitem)
print('解析完毕么么哒')
db.close()            

解析完毕么么哒


## 增加参观时间及图片数字段

In [1]:
#读取结果并将结果用生成器方式输出
import os
from bs4 import BeautifulSoup
base='D:\\sharing bikes\\TA Crawl\\review'
listdir=os.listdir(base)
def getpagedata():
    for dir in listdir:
        skey=dir
        listfile=os.listdir(base+'\\'+dir)
        if listfile:
            for file in listfile:
                with open(base+'\\'+dir+'\\'+file,'rb') as f:              
                    yield {skey:f.read()}

In [3]:
#连接数据库并创建数据表
#创建数据表
import pymysql
db=pymysql.connect(host='localhost',user='root',password='267708',port=3306,db='spider')
cursor=db.cursor()
sql='''
create table if not exists add_item(
attraction_id varchar(30) not null,#景点id
comment_id varchar(30) not null,#评论id
review_time varchar(50),#改评论时间
visit_time varchar(50),#游览时间
picture_count varchar(20))#照片数
'''
cursor.execute(sql)
db.close()

In [4]:
import pymysql
db=pymysql.connect(host='localhost',user='root',password='267708',port=3306,db='spider')
cursor=db.cursor()
def save_to_database(data):
    table='add_item'
    keys=', '.join(data.keys())
    values=', '.join(['%s']*len(data))
    sql='insert into {table}({keys}) values ({values})on duplicate key update'.format(table=table,keys=keys,values=values)
    update=','.join([" {key}=%s".format(key=key) for key in data])
    sql+=update
    try:
        if cursor.execute(sql,tuple(data.values())*2):
            print('successful')
            db.commit()
    except:
        print('Failed')
        db.rollback()

In [8]:
import re
from bs4 import BeautifulSoup
for data in getpagedata():
    for key,value in data.items():
        soup=BeautifulSoup(value,'lxml')
        attraction=key
        #只能嵌套查询，因为有些字段缺失的原因
        rec=re.compile('(\d+)')
        datapart=soup.find_all(id=re.compile('expanded_review_'))
        for data in datapart:
            attraction_id=attraction
            commentid=re.search('UR(\d+)',data.a['name']).group(1)
            review_time=data.find(class_="ratingDate relativeDate")
            if review_time:
                review_time=review_time['title']
            else:
                review_time=data.find(class_="ratingDate").string
                review_time=re.search('Reviewed (.*?)\n$',review_time).group(1)
            visited_time=data.find(class_="recommend-titleInline")
            if visited_time:
                visited_time=visited_time.string
            picture_count=data.find(class_="photosInline")
            if picture_count:
                picture_count=len(picture_count.ul.li)
            dataitem={
            'attraction_id':attraction_id,
            'comment_id':commentid,
            'review_time':review_time,
            'visit_time':visited_time,
            'picture_count':picture_count}
            save_to_database(dataitem)
print('解析完毕么么哒')
db.close()    
            

successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful

successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful

successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful

successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful

successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful

...

successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful

successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful

successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful

successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful

successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful
successful

# 数据读取并保存

In [ ]:
import pymysql
import pandas as pd
db=pymysql.connect(host='localhost',user='root',password='267708',port=3306,db='spider')
cursor=db.cursor()
sql='''
select * from ta_comment1
'''
cursor.execute(sql)
result=cursor.fetchall()
db.close()
result1=pd.DataFrame(result)

In [ ]:
import pandas as pd
result1.to_csv('d:\\...')

# 测试解析程序


In [ ]:
#读取结果并将结果用生成器方式输出
import os
from bs4 import BeautifulSoup
base='D:\\sharing bikes\\TA Crawl\\review'
listdir=os.listdir(base)
def getpagedata():
    for dir in listdir:
        skey=dir
        listfile=os.listdir(base+'\\'+dir)
        if listfile:
            for file in listfile:
                with open(base+'\\'+dir+'\\'+file,'rb') as f:              
                    yield {skey:f.read()}

In [3]:
with open('D:\\sharing bikes\\TA Crawl\\review\\103238\\103238_1','rb') as f: 
    datax=f.read()
import re
from bs4 import BeautifulSoup
soup=BeautifulSoup(datax,'lxml')
rec=re.compile('(\d+)')
datapart=soup.find_all(id=re.compile('expanded_review_'))
for data in datapart:
            commentid=re.search('UR(\d+)',data.a['name']).group(1)
            username=data.find(class_=re.compile('expand_inline scrname mbrName'))
            if username:
                username=username.string.strip()
            location=data.find(class_=re.compile('location'))
            if location:
                userlocation=location.string
            userlevel=data.find(class_=re.compile('bold badge lvl_'))
            if userlevel:
                userlevel=userlevel['class'][2][-2:]
            user_review_counts=data.find(class_="reviewerBadge badge")
            if user_review_counts:
                user_review_counts=user_review_counts.span.string
                user_review_counts=re.search(rec,user_review_counts).group(1)
            user_attrction_review_counts=data.find(class_="contributionReviewBadge badge")
            if user_attrction_review_counts:
                user_attrction_review_counts=user_attrction_review_counts.span.string
                user_attrction_review_counts=re.search(rec,user_attrction_review_counts).group(1)
            user_votes=data.find(class_="helpfulVotesBadge badge no_cpu")
            if user_votes:
                user_votes=user_votes.span.string
                user_votes=re.search(rec,user_votes).group(1)
            review_time=data.find(class_="ratingDate relativeDate")
            if review_time:
                review_time=review_time['title']
            else:
                review_time=data.find(class_="ratingDate").string
                review_time=re.search('Reviewed (.*?)\n$',review_time).group(1)
            score=data.find(class_=re.compile("ui_bubble_rating bubble_"))['class']
            comment_topic=data.find(class_="quote")
            if comment_topic:
                comment_topic=comment_topic.a.span.string
            comment_content=data.find(name='p').get_text('','<br>')
            if comment_content:
                comment_content=comment_content.strip()
            comment_thanks=data.find(class_="numHlp")
            if comment_thanks:
                comment_thanks=comment_thanks.span.string.strip()
                comment_thanks=re.search(rec,comment_thanks).group(1)   
            visited_time=data.find(class_="recommend-titleInline")
            if visited_time:
                visited_time=visited_time.string
            picture_count=data.find(class_="photosInline")
            if picture_count:
                picture_count=len(picture_count.ul.find_all(name='li'))
            dataitem={
            'comment_id':commentid,
            'user_name':username,
            'user_location':userlocation,
            'user_level':userlevel,
            'user_review_counts':user_review_counts,
            'user_attrction_review_counts':user_attrction_review_counts,
            'user_votes':user_votes,
            'review_time':review_time,
            'score':int(float(re.search(rec,score[1]).group(1)))/10,
            'comment_topic':comment_topic,
            'comment_content':comment_content,
            'comment_thanks':comment_thanks,
            'visited_time':visited_time,    
            'picture_count':picture_count
            }
            print(dataitem)

{'comment_id': '629884158', 'user_name': '1010rene', 'user_location': '\r\nMinneapolis, Minnesota\r\n', 'user_level': '02', 'user_review_counts': '9', 'user_attrction_review_counts': '6', 'user_votes': None, 'review_time': 'October 31, 2018', 'score': 5.0, 'comment_topic': 'Must see!', 'comment_content': 'I highly recommend this excursion while in Chicago. The sky deck has enormous windows on all four sides of the building, allowing one to see 50 miles, on a clear day, in any direction. There are glass enclosed decks people can enter to also see the ground below. You will find great photo opportunities here of the entire city of Chicago and beyond! After entering, there are lines to get to the Skydeck and additional lines to get to the glass decks. I would imagine the wait could get very long at peak times.', 'comment_thanks': None, 'visited_time': None, 'picture_count': None}
{'comment_id': '629852201', 'user_name': 'wooleynuts969', 'user_location': '\r\nStoke-on-Trent, United Kingdom

In [ ]:
import re
from bs4 import BeautifulSoup
for data in getpagedata():
    for key,value in data.items():
        soup=BeautifulSoup(value,'lxml')
        attraction=key
        #只能嵌套查询，因为有些字段缺失的原因
        rec=re.compile('(\d+)')
        datapart=soup.find_all(id=re.compile('expanded_review_'))
        for data in datapart:
            attraction_id=attraction
            commentid=re.search('UR(\d+)',data.a['name']).group(1)
            username=data.find(class_=re.compile('expand_inline scrname mbrName'))
            if username:
                username=username.string.strip()
            location=data.find(class_=re.compile('location'))
            if location:
                userlocation=location.string
            userlevel=data.find(class_=re.compile('bold badge lvl_'))
            if userlevel:
                userlevel=userlevel['class'][2][-2:]
            user_review_counts=data.find(class_="reviewerBadge badge")
            if user_review_counts:
                user_review_counts=user_review_counts.span.string
                user_review_counts=re.search(rec,user_review_counts).group(1)
            user_attrction_review_counts=data.find(class_="contributionReviewBadge badge")
            if user_attrction_review_counts:
                user_attrction_review_counts=user_attrction_review_counts.span.string
                user_attrction_review_counts=re.search(rec,user_attrction_review_counts).group(1)
            user_votes=data.find(class_="helpfulVotesBadge badge no_cpu")
            if user_votes:
                user_votes=user_votes.span.string
                user_votes=re.search(rec,user_votes).group(1)
            review_time=data.find(class_="ratingDate relativeDate")
            if review_time:
                review_time=review_time['title']
            else:
                review_time=data.find(class_="ratingDate").string
                review_time=re.search('Reviewed (.*?)\n$',review_time).group(1)
            score=data.find(class_=re.compile("ui_bubble_rating bubble_"))['class']
            comment_topic=data.find(class_="quote")
            if comment_topic:
                comment_topic=comment_topic.a.span.string
            comment_content=data.find(name='p').get_text('','<br>')
            if comment_content:
                comment_content=comment_content.strip()
            comment_thanks=data.find(class_="numHlp")
            if comment_thanks:
                comment_thanks=comment_thanks.span.string.strip()
                comment_thanks=re.search(rec,comment_thanks).group(1)   
            visited_time=data.find(class_="recommend-titleInline")
            if visited_time:
                visited_time=visited_time.string
            picture_count=data.find(class_="photosInline")
            if picture_count:
                picture_count=len(picture_count.ul.find_all(name='li'))
            dataitem={
            'attraction_id':attraction_id,
            'comment_id':commentid,
            'user_name':username,
            'user_location':userlocation,
            'user_level':userlevel,
            'user_review_counts':user_review_counts,
            'user_attrction_review_counts':user_attrction_review_counts,
            'user_votes':user_votes,
            'review_time':review_time,
            'score':int(float(re.search(rec,score[1]).group(1)))/10,
            'comment_topic':comment_topic,
            'comment_content':comment_content,
            'comment_thanks':comment_thanks,
            'visited_time':visited_time,    
            'picture_count':picture_count
            }
            save_to_database(dataitem)
print('解析完毕么么哒')
db.close()            